# News Feed Auto Tagging using Random Forest Classifier

This notebook demonstrates how to preprocess text data of news feed and train Random Forest Classifier to auto tag news feed.

In [27]:
import time
import pandas as pd
import numpy as np

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score

import re
from bs4 import BeautifulSoup
import jieba

## 1. Text Preprocessing

First, let us load the data, preprocess the text data and split the data into training set and test set. For text preprocessing, we build our own stopword dictionary (specific for chinese characters) and perform some standard text cleaning procedures.

* Remove HTML tag using BeautifulSoup
* Remove all english letters, numbers, symbols using regular expression
* Perform chinese word segmentation using Jeiba (a NLP library for chinese language)
* Remove stopwords

In [19]:
# load data
df = pd.read_csv("data/newsfeed.csv",  header=0)
X = df['text']
y = df['tags']

In [20]:
def build_stopwords(filepath):
    '''
    Load customized stopwords.txt
    Return a dictionary of stopwords
    '''
    with open(filepath,'r') as file:
        return set([line.strip() for line in file])

In [21]:
def clean_text(text_array):
    '''
    Remove html tags, symbols, letters a-z A-Z, numbers
    Perform chinese word segmentation
    Remove stopwords 
    Return list of cleaned text
    '''
    cleaned_text_array = []
    for i in range(len(text_array)):
        raw_text = text_array[i]
        cleaned_text = BeautifulSoup(raw_text, 'html5lib').get_text()  
        chinese_only = re.sub('[0-9a-zA-Z\xa0\r\n\t\u3000\u2000-\u206F\u2E00-\u2E7F\!#$%&()*+,\-.\/:;<=>?@\[\]^_`{|}~]','',cleaned_text)
        words = list(jieba.cut(chinese_only, cut_all=False))
        words = [word for word in words if str(word) not in stopwords]
        cleaned_text_array.append(' '.join(words))

    return cleaned_text_array

In [22]:
# build dictionary of stopwords from customized stopwords file
stopwords = build_stopwords(filepath='data/stopwords.txt')

# text preprocessing
X_cleaned = clean_text(X)

print('Before text preprocessing:')
print(X[0])
print('\nAfter text preprocessing:')
print(X_cleaned[0])

Before text preprocessing:
利物浦重賽擊敗乙組仔　英足盃過關 英格蘭足總盃第三圈今晨重賽，貴為英超勁旅的利物浦上場被乙組仔埃克斯特尷尬逼和，多獲一次機會的紅軍不敢再有差池。先有近期回勇的「威爾斯沙維」祖阿倫10分鐘開紀錄，加上兩個小將舒爾奧祖，及祖奧迪西拿下半場各入一球，以3比0擊敗對手，總算在主場挽 <p style="text-align: justify;">英格蘭足總盃第三圈今晨重賽，貴為英超勁旅的利物浦上場被乙組仔埃克斯特尷尬逼和，多獲一次機會的紅軍不敢再有差池。先有近期回勇的「威爾斯沙維」祖阿倫10分鐘開紀錄，加上兩個小將舒爾奧祖，及祖奧迪西拿下半場各入一球，以3比0擊敗對手，總算在主場挽回面子，下一圈對手為韋斯咸。</p> <p style="text-align: justify;">另一場英超球隊對壘，今季異軍突起的李斯特城戰至第三圈就宣告畢業。熱刺憑韓國前鋒孫興<U+615C>上半場遠射破網先開紀錄，換邊後此子助攻予中場查迪尼建功，令球隊以兩球輕取李斯特城，第四圈將面對英甲的高車士打。</p>

After text preprocessing:
利物浦 重賽 擊敗 乙組 仔英足 盃 過關   英格蘭足 總 盃 第三圈 今晨 重賽 貴為 英超 勁 旅 利物浦 上場 乙 組仔 埃克斯 特 尷尬 逼 多獲 一次 機會的紅 軍 不敢 再有 差池 先有 近期 回勇 威爾斯沙維 祖阿倫 分鐘 開紀錄 加上 兩個 小將 舒爾奧祖 及祖奧 迪西 拿下 半場 各入 一球 以比擊敗 對 手 總算 主場 挽   英格蘭足 總 盃 第三圈 今晨 重賽 貴為 英超 勁 旅 利物浦 上場 乙 組仔 埃克斯 特 尷尬 逼 多獲 一次 機會的紅 軍 不敢 再有 差池 先有 近期 回勇 威爾斯沙維 祖阿倫 分鐘 開紀錄 加上 兩個 小將 舒爾奧祖 及祖奧 迪西 拿下 半場 各入 一球 以比擊敗 對 手 總算 主場 挽回 面子 下 一圈 對手 為 韋斯咸   另 一場 英超球 隊 對 壘 今季異 軍 突起 李斯特 城戰 至 第三圈 宣告 畢業 熱刺 憑 韓國 前鋒 孫興 上半 場遠射 破 網先 開紀錄 換邊 後 此子 助攻 予中場 查迪尼 建功 令球隊 兩球 輕取 李斯特 城 第 四圈 將面 對 英甲 高車士 打


In [7]:
# label encode the classes
le = LabelEncoder()
y_le = le.fit_transform(y)
print('\nClass labels: ', list(le.classes_))

# train test split
X_train, X_test, y_train, y_test = train_test_split(X_cleaned, y_le, test_size=0.2, random_state=0)
print("{} training examples and {} test cases".format(len(X_train), len(X_test)))

print("\nTag:", y_train[0], "\nText:", X_train[0])


Class labels:  ['梁振英', '美國大選', '足球']
3115 training examples and 779 test cases

Tag: 2 
Text: 中超 足協 新政 引 地震 限上 名 外援 正選須 小將   為 提升 國內 足球 水平 中國足協 確不 遺餘力 今早 足協 更 宣布 年 新 措施 限制 外援 最多 上場 名人 次 而且 必須 至少 派 一名 小將 擔任 正選 英國 太陽報 指出 中國足協 相信 令 不少 歐洲球 會 鬆 一口 氣   今日 月 日 早上 中國足 協在 官網 發表名 為 中國足 協將 對 中 超中甲 聯賽 部分 相關 規程 內容 進行 調整 文章 表示 中超 中甲球會 聯席 工作 會議 上 中超 中甲 代表 中國足協 同意 調整 球季 中超 中甲 聯賽 規程 主要 內容 以下 兩點   今日 月 日 早上 中國足 協在 官網 發表名 為 中國足 協將 對 中 超中甲 聯賽 部分 相關 規程 內容 進行 調整 文章 表示 中超 中甲球會 聯席 工作 會議 上 中超 中甲 代表 中國足協 同意 調整 球季 中超 中甲 聯賽 規程 主要 內容 以下 兩點   太陽報 車路士 可 鬆 一口 氣 普遍 相信 今次 足協 新政 主要 眼見 中超 中甲球會 天價 買入 外援 卻 無助 提升 國家隊 水平 情況 希望 鼓勵 球會 多 起用 本土 球員 加強 青訓 不過 內 地 網民 擔心 新政 會令國 內球員 身價 暴漲 並打擊 球隊 引入 亞洲 外援 積極性 新政 亦 恐怕 危及 基藍馬 積施利 等 北 漂港將 位置 始終 港將 世界 級 外援 實力 一段 距離 另外 英國 太陽報 認為 中國足協 新政 下 車路士 主帥 干 地將 不用 擔心迪亞 高哥斯達 投 可 專心 爭奪 英超 冠 軍


## 2. Create Bag of Words using TF-IDF

We use `TfidfVectorizer()` from sklearn to convert a text into a numeric vector. In gist, tf-idf transformation counts word occurance frequency and normalizes by putting more weight on rare and meaningful words. After tf-idf transformation, each text is now represented by a feature vector of 1147 dimensions.

In [12]:
# create Bag of Words using tf-idf transform
tfidf = TfidfVectorizer(tokenizer=lambda x: x.split(), lowercase=False, min_df=100) #minumum document frequency set to 100
X_train_tfidf = tfidf.fit_transform(X_train)

print("Number of words in dictionary : {}".format(len(tfidf.get_feature_names())))
print("Example of words in dictionary : \n", tfidf.get_feature_names()[::100])

Number of words in dictionary : 1147
Example of words in dictionary : 
 ['一份', '今日', '兼', '同時', '完場', '應', '方面', '比利', '相關', '自己', '較', '隊']


In [23]:
X_train_tfidf.shape

(3115, 1147)

## 3. Train Random Forest Classifier
Next, we fit Random Forest Classifier on the training set using grid search on 10-fold CV.

In [34]:
# grid search for best paramter set of random forest classifier
rf = RandomForestClassifier()
params = {"max_depth":[None, 10, 50],  #max depth of trees
          "n_estimators":[10, 100]  #no. of tress to ensemble
         } 

model = GridSearchCV(estimator=rf, param_grid=params, scoring='accuracy', cv=10, n_jobs=-1, verbose=0)
model.fit(X_train_tfidf, y_train)

print("\nThe best paramenter set is:", model.best_params_)
print("Mean test score: ", model.cv_results_['mean_test_score'])
print("Std test score: ", model.cv_results_['std_test_score'])


The best paramenter set is: {'max_depth': None, 'n_estimators': 100}
Mean test score:  [0.98812199 0.99165329 0.98298555 0.98844302 0.98619583 0.99133226]
Std test score:  [0.00760298 0.00481237 0.00644868 0.00750141 0.00720326 0.00432309]


## 4. Evaluate on Test Set

Finally, we evaluate the model performance on the 20% test set. 

In [51]:
# make prediction on test set
pred = model.predict(tfidf.transform(X_test))

print('Accuracy: ', accuracy_score(y_test, pred))
print('\n', classification_report(y_test, pred))

Accuracy:  0.9961489088575096

              precision    recall  f1-score   support

          0       0.99      0.99      0.99       174
          1       0.99      0.99      0.99       163
          2       1.00      1.00      1.00       442

avg / total       1.00      1.00      1.00       779

